# Exercise 2-1 (Pytorch MNIST)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import numpy as np

from matplotlib import pyplot as plt
from IPython import display
%matplotlib inline
display.set_matplotlib_formats('svg')

In [14]:
bs=64
epochs=10
num_classes = 10
num_workers = 4
lr=0.1

def load_data(bs, num_workers):
    trans = transforms.ToTensor()
    train_data = datasets.MNIST(root='./data/', train=True, download=True, transform=trans)
    test_data = datasets.MNIST(root='./data/', train=False, download=True, transform=trans)
    
    train_loader = DataLoader(dataset=train_data, batch_size=bs, num_workers=num_workers, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=bs, num_workers=num_workers, shuffle=True)
    
    return train_loader, test_loader

class Net(nn.Module):

    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),

        )
        self.fc = nn.Sequential(
            nn.Linear(7*7*64, 1024),
            nn.Linear(1024, num_classes),

        )

    def forward(self,x):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)

        return output
    
   

In [16]:
def accuracy(y_hat, y):
#     import pdb
#     pdb.set_trace()
    pred = (y == y_hat.argmax(dim=1)).float()
    mean_acc = pred.mean().item()
    
    pred[pred <1] = -1
    return pred.int()*(y+1), mean_acc

def evaluate(net, loader):
    accs = []
    net.eval()
    for idx, data in enumerate(loader):
        img, label = data
        output = net(img)
        corr_label, mean_acc = accuracy(output, label)
        accs.append(mean_acc)
        
    return np.mean(accs)

In [20]:
def train(epochs, lr, bs, num_classes, num_workers):
    net=Net()
    train_loader, test_loader = load_data(bs, num_workers)
    print(net)
    loss = nn.CrossEntropyLoss()
#     loss = nn.NLLLoss()
    optim = torch.optim.SGD(net.parameters(), lr = lr)
    
    train_loss, train_acc, test_acc, acc_per_cls = [], [], [], []
    for epo in range(epochs):
        epo_loss, epo_acc = [], []
        sum_img = np.array([0] * num_classes)
        correct_img = np.array([0] * num_classes)
        net.train()
        for idx, data in enumerate(train_loader):
            img, label = data
# because my Mac doesn't have GPU, I didn't write gpu version.
#             import pdb
#             pdb.set_trace()
            output = net(img)
            optim.zero_grad()
            iter_loss = loss(output, label)
            iter_loss.backward()
            optim.step()
#             import pdb
#             pdb.set_trace()
            corr_label, mean_acc = accuracy(output, label)
            for i in label:
                sum_img[i] += 1
                
            for i in corr_label:
                if i >= 0:
                    correct_img[i-1] += 1

            epo_loss.append(iter_loss.item())
            epo_acc.append(mean_acc)
            
        train_loss.append(np.mean(epo_loss))
        train_acc.append(np.mean(epo_acc))
        test_acc.append(evaluate(net, test_loader))
        acc_per_cls.append(list(correct_img/sum_img))
        
#         import pdb
#         pdb.set_trace()
        
        print(f'Epoch {epo} train loss: {train_loss[epo]}, train acc: {train_acc[epo]}, test acc: {test_acc[epo]}')
        print(f'Acc per class: {acc_per_cls[epo]} \n')

    return train_loss, train_acc, test_acc, acc_per_cls

train_loss, train_acc, test_acc, acc_per_cls = train(epochs, lr, bs, num_classes, num_workers)


100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw
Using downloaded and verified file: ./data/MNIST/raw/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw
Using downloaded and verified file: ./data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Using downloaded and verified file: ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!
Net(
  (convnet): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_fea

KeyboardInterrupt: 

In [31]:
def visual(train_loss, train_acc, test_acc):
    x = range(len(train_loss))
    plt.plot(x, train_loss, label='train_loss')
    plt.plot(x, train_acc, label='train_acc')
    plt.plot(x, test_acc, label='test_acc')
    
    plt.legend()
    plt.show()
visual(train_loss, train_acc, test_acc)

NameError: name 'train_loss' is not defined

# Exercise 2-2 (Tensorflow MNIST)